# Import modules

In [244]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [ ]:
            {crewDetail(crewMemberName: $crew_member_name)
            {
            movieTitle
            movieRevenue
            } 

In [357]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_crew_list = """query data ($nationality: String!, $job: String!)
{crewList(nationality: $nationality, job: $job)
{
aggregateRevenue,
name
}
}"""
query_movie_list = """query data ($crewMemberName: String!)
{movieList(crewMemberName: $crewMemberName)
{
movieTitle,
movieRevenue
}
}
"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_metadata = """query metadata {nationalityList(job:"Director"), jobList(nationality:"Arabic")}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n
queryType {\n      name\n    }\n    mutationType {\n
name\n    }\n    subscriptionType {\n      name\n    }\n
types {\n      ...FullType\n    }\n    directives {\n
name\n      description\n      locations\n      args {\n
...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n
kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n
description\n    args {\n      ...InputValue\n    }\n    type {\n
...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}"""
query_crew_list = json.dumps({'variables': {'nationality': 'Italian', 'job': 'Dialogue Editor'}, 'query': query_crew_list})
query_movie_list = json.dumps({'variables': {'crewMemberName': 'Oliver Roberts'}, 'query': query_movie_list})
query_metadata = json.dumps({'query': query_metadata})
query_introspect = json.dumps({'query': query_introspect})

In [353]:
query_crew_list

'{"variables": {"nationality": "Italian", "job": "Dialogue Editor"}, "query": "query data ($nationality: String!, $job: String!)\\n{crewList(nationality: $nationality, job: $job)\\n{\\naggregateRevenue,\\nname\\n}\\n}"}'

# Test handler functions

In [345]:
import handler 
importlib.reload(handler);

In [358]:
%%time
out = handler.handle(query_movie_list)

An error occurred while resolving field Query.movieList
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/graphql/execution/executor.py", line 447, in resolve_or_error
    return executor.execute(resolve_fn, source, info, **args)
  File "/opt/conda/lib/python3.6/site-packages/graphql/execution/executors/sync.py", line 16, in execute
    return fn(*args, **kwargs)
  File "/labs/bundle/functions/index_by_nationality/handler.py", line 95, in resolve_movie_list
    .groupby(['title'])[['revenue']]
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py", line 6665, in groupby
    observed=observed, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py", line 2152, in groupby
    return klass(obj, by, **kwds)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py", line 599, in __init__
    mutated=self.mutated)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py"

CPU times: user 193 ms, sys: 3.87 ms, total: 197 ms
Wall time: 200 ms


In [361]:
crew_details = (joined_table_index_crew_detail
                        .loc['Oliver Roberts']
#                         .groupby(['title'])[['revenue']]
#                         .sum()
#                         .nlargest(5, 'revenue')
#                         .reset_index()
                       )

TypeError: 'str' object is not callable

In [360]:
handler.joined_table_index_crew_detail.loc['Oliver Roberts']

title          Rogue One: A Star Wars Story
revenue                         1.05606e+09
nationality                         Spanish
job                           Art Direction
Name: Oliver Roberts, dtype: object

In [355]:
out

'{"data": {"crewList": [{"aggregateRevenue": 5555722363.0, "name": "Cheryl Nardi"}, {"aggregateRevenue": 5250610957.0, "name": "Kim Foscato"}, {"aggregateRevenue": 2576362908.0, "name": "Claire Sanfilippo"}, {"aggregateRevenue": 1174630739.0, "name": "Vanessa Lapato"}, {"aggregateRevenue": 782090017.0, "name": "Louis Cerborino"}]}, "errors": null}'

In [356]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.gql'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [177]:
function_url = 'https://labs.cloud.contiamo.com/b0a002bf-d166-5ccc-a552-ebe89882cb90/v1/function/be852eaa-c701-4f73-8ed7-aceb6dd4aeb8/index_by_nationality'

In [337]:
%%time
deployed_output = requests.post(function_url,
                                data=query_movie_list).json()

CPU times: user 23 ms, sys: 0 ns, total: 23 ms
Wall time: 10.2 s


In [338]:
deployed_output

{'errors': ['Server returned unexpected status code: 502 - '],
 'fieldErrors': {}}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True